## Aim
Implement a Multilayer Perceptron (MLP) model for classification:(I) From scratch using NumPy (II) Using Scikit-learn library
and to perform hyperparameter tuning with Grid Search and Cross-validation.

## Theory

## Code



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load dataset
X, y = load_breast_cancer(return_X_y=True)
X = StandardScaler().fit_transform(X)
y = y.reshape(-1, 1)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize parameters
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 1
lr = 0.01
epochs = 500

np.random.seed(42)
W1 = np.random.randn(input_dim, hidden_dim) * 0.01
b1 = np.zeros((1, hidden_dim))
W2 = np.random.randn(hidden_dim, output_dim) * 0.01
b2 = np.zeros((1, output_dim))

# Activation functions
def sigmoid(z): return 1 / (1 + np.exp(-z))
def sigmoid_derivative(a): return a * (1 - a)
def relu(z): return np.maximum(0, z)
def relu_derivative(z): return (z > 0).astype(float)

# Training
for epoch in range(epochs):
    # Forward pass
    z1 = np.dot(X_train, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    
    # Compute loss (binary cross-entropy)
    m = y_train.shape[0]
    loss = -np.mean(y_train * np.log(a2 + 1e-9) + (1 - y_train) * np.log(1 - a2 + 1e-9))
    
    # Backpropagation
    dz2 = a2 - y_train
    dW2 = np.dot(a1.T, dz2) / m
    db2 = np.mean(dz2, axis=0, keepdims=True)
    da1 = np.dot(dz2, W2.T)
    dz1 = da1 * relu_derivative(z1)
    dW1 = np.dot(X_train.T, dz1) / m
    db1 = np.mean(dz1, axis=0, keepdims=True)

    # Parameter update
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2

# Prediction
a2_test = sigmoid(np.dot(relu(np.dot(X_test, W1) + b1), W2) + b2)
y_pred = (a2_test > 0.5).astype(int)
print("Accuracy (from scratch):", accuracy_score(y_test, y_pred))


from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Define MLP and hyperparameter grid
mlp = MLPClassifier(max_iter=500, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(32,), (64,), (32,16)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate_init': [0.001, 0.01]
}

# 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(mlp, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train.ravel())

print("Best Parameters:", grid_search.best_params_)
print("Best CV Accuracy:", grid_search.best_score_)

# Evaluate on test data
best_mlp = grid_search.best_estimator_
y_pred = best_mlp.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


## Result


## Learning Outcome